In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import shap

In [ ]:
data_path = 'data/'

In [ ]:
df = pd.read_csv(os.path.join(data_path, 'data.csv'))
df['Bankrupt?'] = df['Bankrupt?'].astype(bool)
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(df[df.columns[1:]])
y = df['Bankrupt?'].to_numpy()
df_normalized = pd.DataFrame(np.column_stack((y, X)), columns=df.columns)

In [ ]:
df_normalized.describe()

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(df_normalized.corr(numeric_only=True))
plt.title('Correlation plot between columns')
plt.show()

In [ ]:
pca = PCA(10)
pca.fit(df_normalized)

In [ ]:
pd.DataFrame(pca.components_.T, index=df.columns)

In [ ]:
important_cols = ['Bankrupt?', 'Current Liability to Equity', 'Net Income to Stockholder\'s Equity', 'Debt ratio %', 'ROA(A) before interest and % after tax', 'Persistent EPS in the Last Four Seasons', 'Equity to Long-term Liability', 'Current Assets/Total Assets', 'Working Capital to Total Assets', 'Current Liability to Current Assets', 'Pre-tax net Interest Rate', 'Cash Reinvestment %', 'CFO to Assets', 'Net worth/Assets', 'Operating Gross Margin', 'Gross Profit to Sales', 'Working capitcal Turnover Rate', 'Cash Flow to Sales', 'After-tax Net Profit Growth Rate', 'Net Income to Total Assets', 'Net Value Per Share (A)']

In [ ]:
plt.figure(figsize=(12, 12))
sns.heatmap(df_normalized[important_cols].corr(numeric_only=True), annot=True, fmt='.2f')
plt.title('Correlation plot between columns')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Random Forest Classifier

In [ ]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

In [ ]:
y_pred = rf_classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred, normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Random Forest Confusion Matrix")
plt.show()

In [ ]:
y_proba = rf_classifier.predict_proba(X_test)[:, 1]

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Random Forest ROC Curve")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

## Support Vector Machine

In [ ]:
svc = SVC(probability=True)
svc.fit(X_train, y_train)

In [ ]:
y_pred = svc.predict(X_test)

cm = confusion_matrix(y_test, y_pred, normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Support Vector Machine Confusion Matrix")
plt.show()

In [ ]:
y_proba = svc.predict_proba(X_test)[:, 1]

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Support Vector Machine ROC Curve")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

## Logistic Regression

In [ ]:
logr = LogisticRegression(max_iter=1000)

logr.fit(X_train, y_train)
y_pred = logr.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred, normalize='pred')
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Logistic Regression Confusion Matrix")
plt.show()

In [ ]:
y_proba = logr.predict_proba(X_test)[:, 1]

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Logistic Regression ROC Curve")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

This one seems to be the best

# XAI

In [ ]:
# Create a SHAP explainer for a linear (logistic regression) model
explainer = shap.LinearExplainer(logr, X_train)

# Compute SHAP values for the test set
shap_values = explainer(X_test)
shap_values

In [ ]:
bankrupt_predicted_test_idxs = np.argwhere(y_proba > 0.5)
bankrupt_predicted_test_idxs

In [ ]:
y_proba[bankrupt_predicted_test_idxs]

In [ ]:
baseline_logit = explainer.expected_value
baseline_prob = 1 / (1 + np.exp(-baseline_logit))
baseline_logit, baseline_prob

In [ ]:
bankrupt_logit = np.sum(shap_values.values[580]) + baseline_logit
bankrupt_prob = 1 / (1 + np.exp(-bankrupt_logit))
bankrupt_logit, bankrupt_prob

In [ ]:
shap_values_prob = np.zeros((len(shap_values.values), 2))
for i, entry_values in enumerate(shap_values.values):
    val = np.sum(shap_values.values[i])
    shap_values_prob[i, 0] = val
    shap_values_prob[i, 1] = 1 / (1 + np.exp(-val-baseline_logit))

In [ ]:
shap_values_prob[:5]

In [ ]:
xmin = -20; xmax = 20
ymin = 0; ymax = 1
plt.figure(figsize=(14, 8))
plt.hlines([0.5], xmin=xmin, xmax=xmax, colors=['r'], linestyles=['dashed'], label='Threshold')
plt.vlines([-baseline_logit], ymin=ymin, ymax=ymax, colors=['g'], linestyles=['dashed'], label=f'Baseline={-baseline_logit=:.2f}')
plt.scatter(shap_values_prob[:, 0], shap_values_prob[:, 1], label='Test sample probabilities')
plt.title('Shap Values vs probability of test sample')
plt.xticks(range(-20, 26, 5))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Shap value')
plt.ylabel('Probability')
plt.legend()
plt.show()

In [ ]:
print(shap_values)

In [ ]:
# Summary plot (global feature importance)
shap.summary_plot(shap_values, X_test)

In [ ]:
#Plot var: built from column 1 of all shap_values
shap.plots.bar(shap_values)

In [ ]:
i = 0
shap.plots.waterfall(shap_values[i])